## Importing Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.interpolate import interpn
from scipy import ndimage
from sklearn.decomposition import PCA
import time
import multiprocessing as mp
import SimpleITK as sitk
import raster_geometry

## Defining Functions

In [ ]:
def func(x,y,z):
    '''This function interpolates the discrete data into continous data'''
    x1,y1,z1 = np.arange(image_data.shape[0]),np.arange(image_data.shape[1]),np.arange(image_data.shape[2])
    sample_at = [x,y,z]
    d=scipy.interpolate.interpn((x1,y1,z1),image_data,sample_at,method="nearest")
    return d


def getPointOnPlane(s,t,u,v,w):
    '''This function creates the coordinates for a point with given vectors'''
    point=com+w+s*u+t*v
    return point

def getArea(u,v,s,slim):
    '''This function calculates the proportional area with given vectors'''
    area=0
    vec=np.cross(u,v)
    
    par=0.1 
    
    w=par*vec
                        
    for t in range(-slim,slim):
        try:            
            i=getPointOnPlane(s,t,u,v,w)
            j=getPointOnPlane(s,t,u,v,-w)
            
            if(func(i[0],i[1],i[2])==0 and func(j[0],j[1],j[2])==0):
                k=0
            else:
                k=1
        except:
            k=0
        area=area+k
    return area


def RotN(vec,x):
    '''Defining rotation matrix'''
    y= x*np.pi/180
    T=np.array([[vec[0]*vec[0]*(1-np.cos(y))+np.cos(y),vec[0]*vec[1]*(1-np.cos(y))-vec[2]*np.sin(y),vec[0]*vec[2]*(1-np.cos(y))+vec[1]*np.sin(y)],
                [vec[1]*vec[0]*(1-np.cos(y))+vec[2]*np.sin(y),vec[1]*vec[1]*(1-np.cos(y))+np.cos(y),vec[1]*vec[2]*(1-np.cos(y))-vec[0]*np.sin(y)],
                [vec[2]*vec[0]*(1-np.cos(y))-vec[1]*np.sin(y),vec[2]*vec[1]*(1-np.cos(y))+vec[0]*np.sin(y),vec[2]*vec[2]*(1-np.cos(y))+np.cos(y)]])
    return T


def pcascipy(data):
    '''Function that returns pca components'''
    pos=np.where(image_data==1)
    dummy=np.array([[pos[0][i],pos[1][i],pos[2][i]]for i in range(len(pos[0]))])
    pca=PCA()
    pca.fit(dummy)
    return pca.explained_variance_,pca.components_


def normvec(vec):
    '''Calculates the perpendicular vector'''
    T=np.array([[1.074,0,0],[0,1.074,0],[0,0,3]])
    vec_trans=np.dot(T,vec)
    vec_trans_norm=vec_trans/np.linalg.norm(vec_trans)
    T1=np.linalg.inv(T)
    return np.dot(T1,vec_trans_norm)  

def get2dArray(u,v):
    '''Creates array for the cross section image'''
    a=np.array([])
    w=np.array([0,0,0])
    
    for s in range(-20,20):
        for t in range(-20,20):
            try:
                i=getPointOnPlane(s,t,u,v,w)
                b=func(i[0],i[1],i[2])
            except:
                b=0
            a=np.append(a,b)
    return a

def rescale(vec):
    '''Transforms data in physical space'''
    T=np.array([[1.074,0,0],[0,1.074,0],[0,0,3]])
    return T.dot(vec)

## Imports union data, rescales and cuts with bounding box

In [ ]:
image = sitk.ReadImage("Union.mha") 
image_data = sitk.GetArrayFromImage(image)
image_data = np.transpose(image_data, (2, 1, 0))
comalt=np.array(ndimage.measurements.center_of_mass(image_data))
pos = np.where(image_data==1)

pos=rescale(pos)
pos=np.around(pos) 
pos=pos.astype(int)
mask = np.zeros((800,800,800), dtype='int')
for i in range(len(pos[0])):
    mask[pos[0][i],pos[1][i],pos[2][i]]=1
image_data=mask

image_data=scipy.ndimage.morphology.binary_closing(image_data,structure=raster_geometry.sphere(5,3)).astype(int)

img_out = sitk.GetImageFromArray(image_data)
stats=sitk.LabelShapeStatisticsImageFilter()
stats.Execute(img_out)
boundingBox = np.array(stats.GetBoundingBox(1))
image_data=image_data[boundingBox[2]:boundingBox[2]+boundingBox[5],boundingBox[1]:boundingBox[1]+boundingBox[4],boundingBox[0]:boundingBox[0]+boundingBox[3]]

## Imports different artificial volumes

In [ ]:
image_data= raster_geometry.ellipsoid(50,(5,5,20))

In [ ]:
image_data= raster_geometry.ellipsoid(50,(20,10,10)).astype("int")
img_rot = sitk.GetImageFromArray(image_data)
transform = sitk.VersorTransform((1,1,1), np.pi)
outimage=sitk.Resample(img_rot,transform)
image_data=sitk.GetArrayFromImage(outimage)

In [ ]:
image_data= raster_geometry.cuboid(20,(5,10,5))

## The actual iteration to find the maximal plane 

In [ ]:
com = np.array(ndimage.measurements.center_of_mass(image_data))
pca1=pcascipy(image_data)
arr2=[]
arrw=[]
arru=[]

start = time.time()

for i in range(-45,46,9):
    for j in range(-45,46,9):
        for k in range(0,181,10):
            w=pca1[1][np.argsort(pca1[0])[2]]
            u=pca1[1][np.argsort(pca1[0])[1]]
            v=pca1[1][np.argsort(pca1[0])[0]]

            
            w=RotN(u,i).dot(w) 
            u=RotN(u,i).dot(u) 
            v=RotN(u,i).dot(v)
            
            w=RotN(v,j).dot(w) 
            u=RotN(v,j).dot(u) 
            v=RotN(v,j).dot(v)
                   
            
            w=RotN(w,k).dot(w)
            u=RotN(w,k).dot(u)
            v=RotN(w,k).dot(v)
            
            pool = mp.Pool(mp.cpu_count())
            results = pool.starmap(getArea, [(w, u, s,20) for s in range(-20,20)]) 
            results2=sum(results)
            pool.close()        
            arr2.append(results2)
            arrw.append(w)
            arru.append(u)
print(arr2)
end = time.time()
final = end - start
print(final,"Sekunden")
print("Maximaler Flächeninhalt ist",arr2[np.argmax(arr2)],"mit Vektoren: w=",arrw[np.argmax(arr2)],"und u=",arru[np.argmax(arr2)],"p=",com)

## Plots the cross-sectional areas

In [ ]:
x3=np.arange(len(arr2))
y3=arr2
plt.plot(x3,y3,c="black",marker="x",linestyle="-")
plt.ylabel("Area")
plt.xlabel("Plane Index")
plt.savefig("areaplot_cuboid.png")

## Plots cross-section

In [ ]:
u=arru[np.argmax(arr2)]
v=arrw[np.argmax(arr2)]
c=get2dArray(u,v)
c=c.reshape(40,40)
plt.imshow(c, interpolation='nearest',cmap='binary')
plt.ylim(0,40)
plt.xlim(0,40)
plt.savefig("cuboid_Cross-Section.png")

## Plotting 3d points and maximal plane via normal vector

In [ ]:
x, y = np.mgrid[range(int(com[0])-50,int(com[0])+50), range(int(com[1])-50,int(com[1])+50)]
point  = com
normal = np.cross(arru[np.argmax(arr2)],arrv[np.argmax(arr2)])
d=-point.dot(normal)
z = (-normal[0] * x - normal[1] * y - d) * 1. /normal[2]
pos = np.where(image_data==1)
from mayavi.mlab import *
from mayavi import mlab
mlab.clf()
mlab.points3d(pos[0], pos[1], pos[2],color=(0.1,0.1,0.1),scale_factor=1)
mlab.surf(x,y,z,color=(1,1,1))
mlab.show()

In [ ]:
from mayavi.mlab import *
from mayavi import mlab

## Plotting vectors which are used for the iteration

In [ ]:
pca1=pcascipy(image_data)
u=pca1[1][np.argsort(pca1[0])[1]]
v=pca1[1][np.argsort(pca1[0])[0]]
w=pca1[1][np.argsort(pca1[0])[2]]
com = np.array(ndimage.measurements.center_of_mass(image_data))
pos=np.where(image_data==1)


mlab.clf()
mlab.points3d(pos[0], pos[1], pos[2],color=(0.1,0.1,0.1),scale_factor=1.0)
for i in range(-15,16,5):
    for j in range(-15,16,5):
        for k in range(0,181,20):
            w=pca1[1][np.argsort(pca1[0])[2]]
            u=pca1[1][np.argsort(pca1[0])[1]]
            v=pca1[1][np.argsort(pca1[0])[0]]

            
            w=RotN(u,i).dot(w) 
            u=RotN(u,i).dot(u) 
            v=RotN(u,i).dot(v)
            
            w=RotN(v,j).dot(w) 
            u=RotN(v,j).dot(u) 
            v=RotN(v,j).dot(v)
                   
            
            w=RotN(w,k).dot(w)
            u=RotN(w,k).dot(u)
            v=RotN(w,k).dot(v)
            
            
            mlab.quiver3d(com[0],com[1],com[2],w[0],w[1],w[2],scale_factor=30.0)
            mlab.quiver3d(com[0],com[1],com[2],u[0],u[1],u[2],scale_factor=30.0,color=(1,1,1))

        

mlab.show()

## Plotting 3d points + the three principal components

In [ ]:
pca1=pcascipy(image_data)
com = np.array(ndimage.measurements.center_of_mass(image_data))
pos=np.where(image_data==1)

mlab.clf()
mlab.points3d(pos[0], pos[1], pos[2],color=(0.1,0.1,0.1),scale_factor=1.0)
u=pca1[1][np.argsort(pca1[0])[1]] 
v=pca1[1][np.argsort(pca1[0])[0]]
w=pca1[1][np.argsort(pca1[0])[2]] 
mlab.quiver3d(com[0],com[1],com[2],u[0],u[1],u[2],scale_factor=10.0,color=(1,1,1)) #weiß
mlab.quiver3d(com[0],com[1],com[2],v[0],v[1],v[2],scale_factor=10.0,color=(0.75,0.75,0.75)) #grau
mlab.quiver3d(com[0],com[1],com[2],w[0],w[1],w[2],scale_factor=10.0) #rot
mlab.view(focalpoint=com)
mlab.show()

## Plotting the maximal plane + 3d points

In [ ]:
#maximale ebene zeichner
pca1=pcascipy(image_data)
w=pca1[1][np.argsort(pca1[0])[2]]
com = np.array(ndimage.measurements.center_of_mass(image_data))
x, y = np.mgrid[range(int(com[0])-50,int(com[0])+50), range(int(com[1])-50,int(com[1])+50)]
point  = com
u=arrw[np.argmax(arr2)]
v=arru[np.argmax(arr2)]
normal = np.cross(u,v)
d=-point.dot(normal)
z = (-normal[0] * x - normal[1] * y - d) * 1. /normal[2]
pos = np.where(image_data==1)

mlab.clf()
mlab.points3d(pos[0], pos[1], pos[2],color=(0.1,0.1,0.1),scale_factor=1)
mlab.surf(x,y,z,color=(1,1,1))
mlab.view(focalpoint=com)
mlab.show()